In [ ]:
from gerrychain import Graph, Partition, Election
from gerrychain.updaters import Tally, cut_edges
import json
import networkx
from gerrychain import MarkovChain
from gerrychain.constraints import single_flip_contiguous, no_more_discontiguous
from gerrychain.proposals import propose_random_flip
from gerrychain.accept import always_accept
import pandas
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election)
from functools import partial
from gerrychain.proposals import recom
 
from gerrychain.constraints.validity import within_percent_of_ideal_population
import matplotlib.pyplot as plt
import gerrychain 

#NOTE: must have tqdm installed (pip install tqdm in terminal)

In [ ]:
#Use .json of graph 

#NOTE: An island warning will pop up.
#graph = Graph.from_file("/Users/caranix/Documents/MGGG/AK_precincts/alaska_precincts.shp")
#print("graph done")

In [ ]:
graph = Graph.from_json("/Users/caranix/Documents/MGGG/ak_fix.json")

In [ ]:
def parts_adjacency_matrix(partition):
    parts_graph = networkx.Graph()
    for part in partition.parts:
        parts_graph.add_node(part)
    for edge in partition['cut_edges']:
        source, destination = (partition.assignment[node] for node in edge)
        parts_graph.add_edge(source, destination)
    nodelist = sorted(list(partition.parts.keys()))
    return networkx.to_numpy_matrix(parts_graph, nodelist=nodelist)


In [ ]:
#election = gerrychain.Election("AK", ["PRES16", "USH14", "SEN16","GOV18","USH16","USH18"])

In [5]:
elections = [
    Election("PRES16", {"Democratic": "PRES16D", "Republican": "PRES16R"}),
    Election("USH14", {"Democratic": "USH14D", "Republican": "USH14R"}),
    Election("SEN16", {"Democratic": "SEN16D", "Republican": "SEN16R"}),
    Election("GOV18", {"Democratic": "GOV18D", "Republican": "GOV18R"}),
    Election("USH16", {"Democratic": "USH16D", "Republican": "USH16R"}),
    Election("USH18", {"Democratic": "USH18D", "Republican": "USH18R"})
]


In [6]:
#updaters = {"population": gerrychain.updaters.Tally("POPULATION", alias="population"), "AK": election, "cut_edges": gerrychain.updaters.cut_edges}
my_updaters = {"population": updaters.Tally("POPULATION", alias="population")}

election_updaters = {election.name: election for election in elections}
my_updaters.update(election_updaters)

# Create an initial partition
initial_partition = GeographicPartition(graph, assignment="HDIST", updaters=my_updaters)#NOTE: assignment based on House Districts


In [7]:
ideal_population = sum(initial_partition["population"].values()) / len(initial_partition)

proposal = partial(recom,
                   pop_col="POPULATION",
                   pop_target=ideal_population,
                   epsilon=0.05,
                   node_repeats=2
                  )

In [8]:
compactness_bound = constraints.UpperBound(
    lambda p: len(p["cut_edges"]),
    2*len(initial_partition["cut_edges"])
)

In [ ]:
#THIS CHAIN IS  WORKING!! (but it isn't based on proposal, but propose_random_flip)


#chain = gerrychain.MarkovChain(
#    proposal=propose_random_flip,
#    constraints=[constraints.within_percent_of_ideal_population(initial_partition, .03), no_more_discontiguous, compactness_bound],
#    accept=always_accept,
#    initial_state=initial_partition,
#    total_steps=1000
#)


In [9]:
#THIS CHAIN does work! (thanks Daryl) 

chain = MarkovChain(
    proposal=proposal, 
    constraints=[
        constraints.within_percent_of_ideal_population(initial_partition, .05),
        compactness_bound, no_more_discontiguous
    ],
    accept=accept.always_accept,
    initial_state=initial_partition,
    total_steps=1000
)

#NOTE: The current Alaksa plan does not have population within 2%, I increased it to 3% population deviation.

#NOTE: Apparently, Alaska is not single flip contiguous? Islands might be the issue? 
#Changed to no_more_discontiguous, so it doesn't get any worse. 


In [13]:
chain_results = {"PRES16": [],
                 "USH14": [],
                 "SEN16": [],
                 "GOV18": [],
                 "USH16": [], 
                 "USH18": []}

for partition in chain:
    for key, _ in chain_results.items():
        chain_results[key].append(sorted(partition[key].percents("Democratic"))) 


In [14]:
chain_dfs = {"PRES16": [],
             "USH14": [],
             "SEN16": [],
             "SEN18": [],
             "USH16": [], 
             "USH18": []}

for key, val in chain_results.items():
    chain_dfs[key] = pandas.DataFrame(val)

In [12]:
matrix = []
allAssignments = {0: chain.state.assignment}
for step in chain:
        #allAssignments[chain.counter + 1] = [step.flips]
        allAssignments[chain.counter] = step.assignment
        if (chain.counter % 100 == 0):
            print(chain.counter)
            matrix.append((parts_adjacency_matrix(step).tolist()))
with open('AK_output1000_test.json', "w") as m:
    m.write(json.dumps(matrix))
print("Successfully created AK_output1000.json")


100
200
300
400
500
600
700
800
900
1000
Successfully created AK_output1000.json


In [ ]:
for i in range(1000):
    with open("./Assignments/AK_House_"+str(i)+".json",'w') as wf:
        json.dump(dict(allAssignments[i]), wf)

In [ ]:
#data = json.dumps(networkx.readwrite.json_graph.adjacency_data(graph))
#It won't let me save graph due to a Polygon error- not a major issue for me atm 
#with open('/Users/caranix/Documents/alaska_graphAK_DATA.json') as f:
#        data = json.load(f)
#graph = networkx.readwrite.json_graph.adjacency_graph(data)
#this is for when I can fix saving the grpah as a .json

In [ ]:
#allAssignments[1]

In [ ]:
#dict(step.assignment)


In [ ]:
chain.counter


In [ ]:
allAssignments

In [ ]:
import geopandas as gpd

In [ ]:
df = gpd.read_file("/Users/caranix/Documents/MGGG/AK_precincts/alaska_precincts.shp")

In [ ]:
for i in range(1000):
    df["plan"+str(i)] = df.index.map(dict(allAssignments[i]))

In [ ]:
plt.figure()
df.plot(column="plan200", cmap = "hsv")
plt.axis('off')


In [ ]:
for i in range(1000):
    plt.figure()
    df.plot(column="plan"+str(i), cmap = "hsv")
    plt.axis('off')
    plt.savefig("./Pictures/AK_plot_"+str(i)+".png")  
    plt.close()
    
    
    

In [ ]:
new_part = GeographicPartition(graph, assignment=df["plan11"], updaters=my_updaters)#NOTE: assignment based on House Districts


In [ ]:
chain_results = {"PRES16": [],
                 "USH14": [],
                 "SEN16": [],
                 "GOV18": [],
                 "USH16": [], 
                 "USH18": []}

for key in chain_results.keys():
    chain_results[key].append(sorted(new_part[key].percents("Democratic"))) 
